In [2]:
from dotenv import load_dotenv
load_dotenv()

True

# LLM

In [4]:
from langchain_ollama import ChatOllama

local_llm = "llama3.2:latest"
llm = ChatOllama(model=local_llm, temperature=0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0, format="json")

# User Input

In [5]:
input = {
    "search_queries": [
        "High-level overview of intelligent user interfaces and their impact on modern UI/UX design",
        "Tools and frameworks for building intelligent user interfaces: A 2024 guide"
    ],
    "specific_questions": [
        "What are the latest trends in intelligent user interfaces, and how are they shaping user experience?",
        "What are the best practices for ensuring accessibility and inclusivity in AI-powered user interfaces?",
        "What is a woman?"
    ],
    "platform": "google",
    "time_horizon": 185, 
    "max_outputs": 5
}

### Tracing 

Optionally, use [LangSmith](https://www.langchain.com/langsmith) for tracing. 

In [11]:
import os
import getpass


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "local-llama32-rag"

# Run the graph